In [1]:
import  torch
from    torch_geometric.data import Data

/Users/shervin/opt/anaconda3/envs/AIDAVA/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.chdir('..') # set directoroy to AIDAVA-KGE-Framework (run once)

In [3]:
import  pickle 

from    src.rgnn import RGCNEncoder

import  torch.nn.functional as F
import  tqdm

### Load and prepare data

In [4]:
data_path   = './data/m2skg/'
filename    = 'relation_train.pkl'

node_embed_dim  = 100

In [5]:
with open(data_path+filename,'rb') as f:
    data = pickle.load(f)

In [6]:
nodes = dict({})
node_count = 0 

relations = dict({})
rel_count = 0 

triples = []

for h,r,t in data:

    if h not in nodes.keys():
        nodes[h] = node_count
        node_count += 1

    if t not in nodes.keys():
        nodes[t] = node_count
        node_count += 1

    if r not in relations.keys():
        relations[r] = rel_count 
        rel_count += 1 

    triples.append( ( nodes[h],relations[r] , nodes[t] ) )




In [14]:
rel_count , node_count, len(triples)

(9, 3574, 10133)

In [16]:
triples[0]

(0, 0, 1)

### TransE for Link Prediction

In [8]:
from    src.TransE import TransE

In [9]:
device = torch.device('cpu')

In [10]:
trans_e = TransE(num_entities = node_count,
       num_relations = rel_count,
       device = device)

In [11]:
trans_e._train(triples);

/Users/shervin/Documents/AIDAVA/aidava-kge-framework/src/loaders.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.edges_index[idx][0]), self.edges_index[idx][1],   torch.tensor(self.edges_index[idx][2]), torch.tensor(neg_sample[0][0]) , torch.tensor(neg_sample[1]) , torch.tensor(neg_sample[0][1])


epoch 0,	 train loss 0.99
epoch 10,	 train loss 0.73
epoch 20,	 train loss 0.63
epoch 30,	 train loss 0.58
epoch 40,	 train loss 0.56
epoch 50,	 train loss 0.53
epoch 60,	 train loss 0.52
epoch 70,	 train loss 0.52
epoch 80,	 train loss 0.51
epoch 90,	 train loss 0.50


In [12]:
trans_e._eval(triples)

hits@1 0.47 hits@10 0.89  MR 4.78 MRR  0.61
